# Programa de Cosecha • Metaheurísticas (GA, SA, TS)

Este notebook ejecuta el flujo completo: generación de datos, evaluación y las tres metaheurísticas.
El costo que se grafica es el **costo penalizado a minimizar**, por lo que **debe disminuir**.

In [ ]:
import sys
from pathlib import Path
# Asegura que el directorio del proyecto (padre de 'notebooks/') esté en sys.path
PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from core.model import generate_problem, evaluate, greedy_baseline
from core.ga import run_ga
from core.sa import run_sa
from core.tabu import run_tabu
from core.utils import plot_convergence

ARTIFACTS = Path('../artifacts')
ARTIFACTS.mkdir(parents=True, exist_ok=True)

## 1) Parámetros del problema

In [ ]:
seed = 42
N = 80
workers = 10
K = 3
C = 1800

problem = generate_problem(N=N, workers=workers, K=K, C=C, seed=seed)
df = pd.DataFrame({
    'parcela': np.arange(problem.N, dtype=int),
    'Vi_kg': problem.V.astype(int),
    'Pi_horas': problem.P.astype(float),
    'ratio_Vi_Pi': (problem.V / np.maximum(problem.P, 1e-9))
})
df.head()

## 2) Baseline greedy y evaluación

In [ ]:
xg = greedy_baseline(problem)
ev_g = evaluate(problem, xg)
ev_g

## 3) Algoritmo Genético (GA)

In [ ]:
ga_params = dict(pop=80, gens=300, pc=0.8, pm=0.02, elite=1, tournament_k=3)
ga_res = run_ga(problem, ga_params, seed=seed)
fig, _ = plot_convergence(ga_res.history_best_cost, 'GA: Convergencia (Costo penalizado)', 'Generación')
fig.savefig(ARTIFACTS / 'ga_convergence.png', dpi=150)
evaluate(problem, ga_res.best_X)

## 4) Recocido Simulado (SA)

In [ ]:
sa_params = dict(T0=1.0, Tend=1e-3, iters=N*300, alpha=0.99)
sa_res = run_sa(problem, sa_params, seed=seed)
fig, _ = plot_convergence(sa_res.history_best_cost, 'SA: Convergencia (Costo penalizado)', 'Iteración')
fig.savefig(ARTIFACTS / 'sa_convergence.png', dpi=150)
evaluate(problem, sa_res.best_X)

## 5) Búsqueda Tabú (TS)

In [ ]:
ts_params = dict(iters=N*300, tabu_size=int(np.ceil(0.1*N)), consider_all=True)
ts_res = run_tabu(problem, ts_params, seed=seed)
fig, _ = plot_convergence(ts_res.history_best_cost, 'TS: Convergencia (Costo penalizado)', 'Iteración')
fig.savefig(ARTIFACTS / 'ts_convergence.png', dpi=150)
evaluate(problem, ts_res.best_X)

## 6) Comparativa

In [ ]:
rows = []
for name, res, x in [
    ('GA', ga_res, ga_res.best_X),
    ('SA', sa_res, sa_res.best_X),
    ('TS', ts_res, ts_res.best_X),
]:
    ev = evaluate(problem, x)
    rows.append({
        'algoritmo': name,
        'Z': ev['Z'],
        'hours_used': ev['hours_used'],
        'kg_used': ev['kg_used'],
        'feasible': ev['feasible_all'],
        'best_cost': (res.history_best_cost[-1] if hasattr(res, 'history_best_cost') else None),
    })
comp_df = pd.DataFrame(rows)
comp_df

## 7) Exportar resultados

In [ ]:
best_name, best_res = max([('GA', ga_res), ('SA', sa_res), ('TS', ts_res)], key=lambda t: evaluate(problem, t[1].best_X)['Z'])
best_x = best_res.best_X
df_out = df.copy()
df_out['X'] = best_x
df_out.to_csv(ARTIFACTS / 'best_solution.csv', index=False)
(best_name, evaluate(problem, best_x))

## 8) Conclusiones breves

- Se compararon GA, SA y TS sobre un dataset reproducible.
- Se observó descenso del costo penalizado en las tres técnicas, con variaciones en velocidad de convergencia.
- El mejor algoritmo para este dataset es el que obtuvo mayor **Z** factible (ver tabla comparativa); puede variar con `seed`.